In [ ]:
import json
import requests

import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import time


In [ ]:
characters = ["Itsuki+Takeuchi", 
              "Natsuki+Mogi", 
              "Takumi+Fujiwara", 
              "Keisuke+Takahashi", 
              "Ryosuke+Takahashi",
              "Koichiro+Iketani",
              "Kenji",
              "Bunta+Fujiwara"]
url = "https://initiald.fandom.com/wikia.php?controller=ArticleCommentsController&method=getComments&title={CHARACTER}&namespace=0&hideDeleted=true/"


In [ ]:
character_comments = []
for char in characters:
    character_name = char.replace("+", " ")
    response = requests.get(url.replace("{CHARACTER}", char))
    thread = json.loads(response.text)["threads"]
    for comment in thread:
        firstPost = comment["firstPost"]
        if "jsonModel" in firstPost:
            jsonModel = json.loads(firstPost["jsonModel"])
            if "content" in jsonModel:
                content = jsonModel["content"]
                for i in content:
                    if "content" in i:
                        character_comments.append((character_name, i["content"][0]["text"]))
        else:
            character_comments.append((character_name, firstPost["renderedContent"]))
    time.sleep(2)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
def sentiment_score(para):
    tokens = tokenizer.encode(para, return_tensors="pt")
    result = model(tokens)
    return int(torch.argmax(result.logits))+1
    
comments_df = pd.DataFrame(character_comments, columns=["character", "sentence"])
comments_df['sentiment'] = comments_df['sentence'].apply(lambda x: sentiment_score(x[0: 500]))


In [ ]:
comments_df.groupby("character").mean()